In [1]:
import pyodbc
import os
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

def odbc_conn(odbc):
    '''Establishes ODBC connection. User enters in ODBC name. Pyodbc cursor is returned.'''
    connection = pyodbc.connect('DSN=' + odbc, autocommit = True)
    if connection:
        print('Successfully Connected!')
    cursor = connection.cursor()
    return cursor

def py_sql(odbc, sql):
    '''Establishes odbc connection and calls specific sql of user. User enters ODBC name and sql.
    Returns a pandas dataframe.'''
    c = odbc_conn(odbc)
    sql_obj = c.execute(sql)
    #Obtaining field names of table.
    fields = [column[0] for column in c.description]
    #Creating dataframe. Need to convert pyodbc rows to lists.
    row_lists = [list(r) for r in sql_obj]
    df = DataFrame(row_lists, columns = fields)
    return df

def sql_dtypes(df):
    '''Contains common pandas dtypes and maps to SQL dtypes. User enters dataframe and a list of SQL
    dtypes is returned.'''
    dtyp = {'int64': 'INT', 'object': 'VARCHAR', 'float64': 'FLOAT', 'datetime64[ns]': 'DATE'}
    return [dtyp[str(df.iloc[:, i].dtype)] for i in range(len(df.columns))]

def cr_table(df, db, schema, tab_name, c, pk = ''):
    '''Produces and executes sql to create table from dataframe. User has the option to specify a
    primary key. If not specified the variable defaults to an empty string. db is the database.
    schema is the db schema. tab_name is the table name. c is the pyodbc cursor.'''
    sql = 'CREATE TABLE ' + '.'.join([db, schema, tab_name]) + ' ('
    for i, v in enumerate(sql_dtypes(df)):
        beg_sql = sql + '[' + str(df.columns[i]) + '] ' + v
    #If varchar dtype adding varchar length to be max length of value in varchar column.
        if v == 'VARCHAR':
            beg_sql = beg_sql + '(' + str(df[df.columns[i]].map(len).max()) + ')'
        if pk == str(df.columns[i]):
            sql = beg_sql + ' PRIMARY KEY, '
        else:
            sql = beg_sql + ', '
    #Removing last comma and space.
    sql = sql[:-2] + ')'
    c.execute(sql)